In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 35)

In [3]:
# add observation index
data["row_index"] = data.index

In [4]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 36)
test: (39821, 36)
known: (2319611, 36)
unknown: (19918, 36)


In [5]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff', 'hours',
       'is_listened_lag', 'genre_plays', 'genre_skips', 'artist_plays',
       'artist_skips', 'album_plays', 'album_skips', 'song_plays',
       'song_skips', 'user_ratio_flow', 'user_ratio_full', 'genre_ratio',
       'artist_ratio', 'song_ratio', 'platform_name1', 'platform_name2',
       'platform_family1', 'platform_family2', 'row_index'],
      dtype='object')


## 2. MODELING

### 2.1. INITIALIZING

In [6]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([u, s])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
x = Dropout(0.5)(embedding_dense)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([user_in, song_in], output)
model.compile(optimizer = "Adagrad", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [9]:
# run the estimations on training data
model.fit([tr.user_id, tr.media_id], tr.is_listened, 
validation_data = ([ts.user_id, ts.media_id], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2279790 samples, validate on 39821 samples
Epoch 1/10
2279790/2279790 [==============================] - 165s - loss: 0.5350 - acc: 0.7456 - val_loss: 0.6935 - val_acc: 0.6525
Epoch 2/10
2279790/2279790 [==============================] - 154s - loss: 0.4953 - acc: 0.7703 - val_loss: 0.6709 - val_acc: 0.6662
Epoch 3/10
2279790/2279790 [==============================] - 153s - loss: 0.4842 - acc: 0.7760 - val_loss: 0.6459 - val_acc: 0.6665
Epoch 4/10
2279790/2279790 [==============================] - 161s - loss: 0.4762 - acc: 0.7806 - val_loss: 0.6399 - val_acc: 0.6704
Epoch 5/10
2279790/2279790 [==============================] - 142s - loss: 0.4696 - acc: 0.7840 - val_loss: 0.6313 - val_acc: 0.6669
Epoch 6/10
2279790/2279790 [==============================] - 130s - loss: 0.4636 - acc: 0.7876 - val_loss: 0.6361 - val_acc: 0.6677
Epoch 7/10
2279790/2279790 [==============================] - 129s - loss: 0.4578 - acc: 0.7906 - val_loss: 0.6743 - val_acc: 0.6638
Epoch 8/10
227979

In [10]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.user_id, ts.media_id])
pred.to_csv(path + "pred_valid/keras_newdata_flow_song100_10_nonum.csv", index = False)
pred.head(5)

row_index  is_listened
2167       2167     0.986314
2168       2168     0.980918
2240       2240     0.997020
4314       4314     0.930794
4321       4321     0.998955

In [11]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.70609161184184155

### 2.3. SECOND STAGE

In [12]:
# run the estimations on full known data
model.fit([kn.user_id, kn.media_id], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 185s - loss: 0.4438 - acc: 0.7992   
Epoch 2/10
2319611/2319611 [==============================] - 183s - loss: 0.4384 - acc: 0.8014   
Epoch 3/10
2319611/2319611 [==============================] - 162s - loss: 0.4341 - acc: 0.8034   
Epoch 4/10
2319611/2319611 [==============================] - 133s - loss: 0.4302 - acc: 0.8054   
Epoch 5/10
2319611/2319611 [==============================] - 132s - loss: 0.4266 - acc: 0.8071   
Epoch 6/10
2319611/2319611 [==============================] - 132s - loss: 0.4240 - acc: 0.8084   
Epoch 7/10
2319611/2319611 [==============================] - 131s - loss: 0.4216 - acc: 0.8096   
Epoch 8/10
2319611/2319611 [==============================] - 135s - loss: 0.4172 - acc: 0.8116   
Epoch 9/10
2319611/2319611 [==============================] - 109s - loss: 0.4140 - acc: 0.8129   
Epoch 10/10
2319611/2319611 [==============================] - 91s - loss: 0.4128 - acc: 0.8134    


In [13]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.user_id, un.media_id])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_flow_song100_10_nonum.csv", index = False)
pred.head(5)

sample_id  is_listened
2332218          0     0.994443
2182792          1     0.749971
2092769          2     0.515415
1806368          3     0.357551
2018166          4     0.990946